In [ ]:
import pandas as pd
import numpy as np
import math
import string
from collections import defaultdict

In [ ]:
with open(r"/content/drive/MyDrive/HMMTraining.pos", 'r') as f:
  training_corpus = f.readlines()

training_corpus[0:5]

['In\tIN\n', 'an\tDT\n', 'Oct.\tNNP\n', '19\tCD\n', 'review\tNN\n']

In [ ]:
with open(r"/content/drive/MyDrive/HMMTest.pos", 'r') as f:
  test_corpus = f.readlines()

test_corpus[0:5]

['The\tDT\n', 'economy\tNN\n', "'s\tPOS\n", 'temperature\tNN\n', 'will\tMD\n']

In [ ]:
with open(r"/content/drive/MyDrive/HMM_vocab.txt", 'r') as f:
  voc_l = f.read().split('\n')
voc_l[0:5]

['!', '#', '$', '%', '&']

In [ ]:
#Creating a dict for voc_l list

vocab = dict(zip(sorted(voc_l), range(len(sorted(voc_l)))))
vocab

{'': 0,
 '!': 1,
 '#': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 "''": 7,
 "'40s": 8,
 "'60s": 9,
 "'70s": 10,
 "'80s": 11,
 "'86": 12,
 "'90s": 13,
 "'N": 14,
 "'S": 15,
 "'d": 16,
 "'em": 17,
 "'ll": 18,
 "'m": 19,
 "'n'": 20,
 "'re": 21,
 "'s": 22,
 "'til": 23,
 "'ve": 24,
 '(': 25,
 ')': 26,
 ',': 27,
 '-': 28,
 '--': 29,
 '--n--': 30,
 '--unk--': 31,
 '--unk_adj--': 32,
 '--unk_adv--': 33,
 '--unk_digit--': 34,
 '--unk_noun--': 35,
 '--unk_punct--': 36,
 '--unk_upper--': 37,
 '--unk_verb--': 38,
 '.': 39,
 '...': 40,
 '0.01': 41,
 '0.0108': 42,
 '0.02': 43,
 '0.03': 44,
 '0.05': 45,
 '0.1': 46,
 '0.10': 47,
 '0.12': 48,
 '0.13': 49,
 '0.15': 50,
 '0.17': 51,
 '0.19': 52,
 '0.2': 53,
 '0.24': 54,
 '0.25': 55,
 '0.3': 56,
 '0.32': 57,
 '0.375': 58,
 '0.4': 59,
 '0.43': 60,
 '0.45': 61,
 '0.5': 62,
 '0.53': 63,
 '0.59': 64,
 '0.6': 65,
 '0.60': 66,
 '0.7': 67,
 '0.71': 68,
 '0.75': 69,
 '0.8': 70,
 '0.88': 71,
 '0.9': 72,
 '0.94': 73,
 '0.95': 74,
 '1': 75,
 '1,000': 76,
 '1,012': 77,


In [ ]:
prep = ['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from', 'several', '--unk--', 'points', 'this', 'week', 'with', 'readings', 'on', 'trade', 'output', 'housing', 'and', 'inflation']

In [ ]:
test_corpus

['The\tDT\n',
 'economy\tNN\n',
 "'s\tPOS\n",
 'temperature\tNN\n',
 'will\tMD\n',
 'be\tVB\n',
 'taken\tVBN\n',
 'from\tIN\n',
 'several\tJJ\n',
 'vantage\tNN\n',
 'points\tNNS\n',
 'this\tDT\n',
 'week\tNN\n',
 ',\t,\n',
 'with\tIN\n',
 'readings\tNNS\n',
 'on\tIN\n',
 'trade\tNN\n',
 ',\t,\n',
 'output\tNN\n',
 ',\t,\n',
 'housing\tNN\n',
 'and\tCC\n',
 'inflation\tNN\n',
 '.\t.\n',
 '\n',
 'The\tDT\n',
 'most\tRBS\n',
 'troublesome\tJJ\n',
 'report\tNN\n',
 'may\tMD\n',
 'be\tVB\n',
 'the\tDT\n',
 'August\tNNP\n',
 'merchandise\tNN\n',
 'trade\tNN\n',
 'deficit\tNN\n',
 'due\tJJ\n',
 'out\tIN\n',
 'tomorrow\tNN\n',
 '.\t.\n',
 '\n',
 'The\tDT\n',
 'trade\tNN\n',
 'gap\tNN\n',
 'is\tVBZ\n',
 'expected\tVBN\n',
 'to\tTO\n',
 'widen\tVB\n',
 'to\tTO\n',
 'about\tIN\n',
 '$\t$\n',
 '9\tCD\n',
 'billion\tCD\n',
 'from\tIN\n',
 'July\tNNP\n',
 "'s\tPOS\n",
 '$\t$\n',
 '7.6\tCD\n',
 'billion\tCD\n',
 ',\t,\n',
 'according\tVBG\n',
 'to\tTO\n',
 'a\tDT\n',
 'survey\tNN\n',
 'by\tIN\n',
 'M

In [ ]:
for w in training_corpus:
    print(w)
    print("------------")
    break

In	IN

------------


In [ ]:
import string


# Punctuation characters
punct = set(string.punctuation)

# Morphology rules used to assign unknown word tokens
noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]

In [ ]:
def assign_unk(tok):
    """
    Assign unknown word tokens
    """
    # Digits
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Punctuation
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Upper-case
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Nouns
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Verbs
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Adjectives
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Adverbs
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"

    return "--unk--"

In [ ]:
def get_word_tag(line, vocab): 
    
    if not line.split():
        word = "--n--"
        tag = "--s--"
        return word, tag
    else:
        word, tag = line.split()
        if word not in vocab: 
            # Handle unknown words
            word = assign_unk(word)
        return word, tag
    return None 

In [ ]:
# Creating dictionaries for getting transition, emission and tag counts


def create_dictionaries(training_corpus, vocab):

    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    prev_tag = '--s--' 
    i = 0 

    for word_tag in training_corpus:
        i += 1
        word, tag = get_word_tag(word_tag,vocab)
        transition_counts[(prev_tag, tag)] += 1
        emission_counts[(tag, word)] += 1
        tag_counts[tag] += 1
        prev_tag = tag
        
    return emission_counts, transition_counts, tag_counts

In [ ]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

In [ ]:
transition_counts

defaultdict(int,
            {('--s--', 'IN'): 5050,
             ('IN', 'DT'): 32364,
             ('DT', 'NNP'): 9044,
             ('NNP', 'CD'): 1752,
             ('CD', 'NN'): 7377,
             ('NN', 'IN'): 32885,
             ('IN', '``'): 546,
             ('``', 'DT'): 1014,
             ('DT', 'NN'): 38873,
             ('NN', "''"): 686,
             ("''", 'IN'): 591,
             ('IN', 'NNP'): 14753,
             ('NNP', 'POS'): 5094,
             ('POS', 'NNP'): 905,
             ('NNP', 'NNP'): 34465,
             ('NNP', '('): 313,
             ('(', '``'): 42,
             ('``', 'VBN'): 75,
             ('VBN', 'NNS'): 759,
             ('NNS', 'VBP'): 5076,
             ('VBP', 'DT'): 1473,
             ('NNP', ','): 12688,
             (',', "''"): 2796,
             ("''", 'NN'): 242,
             ('NN', 'CC'): 5369,
             ('CC', 'NNS'): 1714,
             ('NNS', ')'): 98,
             (')', ','): 225,
             (',', 'DT'): 6487,
             (',', '

In [ ]:
emission_counts

defaultdict(int,
            {('IN', 'In'): 1735,
             ('DT', 'an'): 3142,
             ('NNP', 'Oct.'): 317,
             ('CD', '19'): 100,
             ('NN', 'review'): 36,
             ('IN', 'of'): 22925,
             ('``', '``'): 6967,
             ('DT', 'The'): 6795,
             ('NN', 'Misanthrope'): 3,
             ("''", "''"): 6787,
             ('IN', 'at'): 4361,
             ('NNP', 'Chicago'): 197,
             ('POS', "'s"): 8079,
             ('NNP', 'Goodman'): 7,
             ('NNP', 'Theatre'): 5,
             ('(', '('): 1153,
             ('VBN', '--unk_upper--'): 93,
             ('NNS', '--unk_upper--'): 350,
             ('VBP', 'Take'): 1,
             ('DT', 'the'): 41098,
             ('NN', 'Stage'): 1,
             ('IN', 'in'): 14957,
             ('NNP', '--unk_upper--'): 4635,
             ('NNP', 'City'): 132,
             (',', ','): 48723,
             ('NN', 'Leisure'): 2,
             ('CC', '&'): 1033,
             ('NNS', 'Arts'): 2,


In [ ]:
tag_counts

defaultdict(int,
            {'#': 142,
             '$': 7372,
             "''": 6919,
             '(': 1366,
             ')': 1376,
             ',': 48727,
             '--s--': 39832,
             '.': 39478,
             ':': 4772,
             'CC': 23947,
             'CD': 36568,
             'DT': 81842,
             'EX': 863,
             'FW': 234,
             'IN': 98554,
             'JJ': 61217,
             'JJR': 3238,
             'JJS': 1947,
             'LS': 36,
             'MD': 9803,
             'NN': 132935,
             'NNP': 91466,
             'NNPS': 2673,
             'NNS': 59856,
             'PDT': 370,
             'POS': 8701,
             'PRP': 17436,
             'PRP$': 8407,
             'RB': 30970,
             'RBR': 1768,
             'RBS': 451,
             'RP': 2662,
             'SYM': 58,
             'TO': 22357,
             'UH': 97,
             'VB': 26438,
             'VBD': 29889,
             'VBG': 14846,
             '

In [ ]:
states  = sorted(tag_counts.keys()) #list of unique POS tags

In [ ]:
def create_transition_matrix(alpha, tag_counts, transition_counts):

    all_tags = sorted(tag_counts.keys())
    num_tags = len(all_tags)
    
    A = np.zeros((num_tags,num_tags))
    trans_keys = set(transition_counts.keys())

    for i in range(num_tags):

        for j in range(num_tags):

            count = 0
            key = (all_tags[i],all_tags[j])

            if key in trans_keys:
                count = transition_counts[key]

            count_prev_tag = tag_counts[all_tags[i]]

            A[i,j] = (count + alpha ) / ( count_prev_tag + alpha * num_tags)
            
    return A

In [ ]:
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)

In [ ]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):

    num_tags = len(tag_counts)
    all_tags = sorted(tag_counts.keys())
    num_words = len(vocab)
    
    B = np.zeros((num_tags, num_words))

    emis_keys = set(list(emission_counts.keys()))
    
    for i in range(num_tags):
        
        for j,word in enumerate(vocab):
            count = 0
            key =  (all_tags[i], word)
            if key in emis_keys:
                count = emission_counts[key]
            count_tag = tag_counts[all_tags[i]]
            B[i,j] = (count + alpha) / (count_tag + alpha * num_words)
    return B

In [ ]:
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))


In [ ]:
float('inf')
float('-inf')

-inf

In [ ]:
def initialize(states, tag_counts, A, B, corpus, vocab):

    num_tags = len(tag_counts)
    best_probs = np.zeros((num_tags, len(corpus)))
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)

    s_idx = states.index("--s--")
    for i in range(num_tags):
        if A[0,i] == 0:
            best_probs[i,0] = float("-inf")
        else:
            best_probs[i,0] = math.log(A[s_idx,i])  +  math.log(B[i,vocab[corpus[0]]])
            
    return best_probs, best_paths

In [ ]:
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)


In [ ]:
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab):

    num_tags = best_probs.shape[0]
    for i in range(1, len(test_corpus)): 
        for j in range(num_tags):
            best_prob_i = float("-inf")
            best_path_i = None

            for k in range(num_tags):
                prob = best_probs[k, i-1] + math.log(A[k,j]) + math.log(B[j, vocab[test_corpus[i]]])
                if prob > best_prob_i:
                    best_prob_i = prob
                    best_path_i = k
            best_probs[j,i] = best_prob_i
            best_paths[j,i] = best_path_i
            
    return best_probs, best_paths

In [ ]:
best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)


In [ ]:
def viterbi_backward(best_probs, best_paths, corpus, states):

    m = best_paths.shape[1]
    z = [None] * m
    
    num_tags = best_probs.shape[0]

    best_prob_for_last_word = float('-inf')

    pred = [None] * m

    for k in range(num_tags):

        if best_probs[k,-1] > best_prob_for_last_word:
            best_prob_for_last_word = best_probs[k,-1]
            z[m - 1] = k
    pred[m - 1] = states[k]
    for i in range(m-1, 0, -1):

        pos_tag_for_word_i = best_paths[z[i], i]
        z[i - 1] = pos_tag_for_word_i
        pred[i - 1] = states[pos_tag_for_word_i]
    return pred

In [ ]:
pred = viterbi_backward(best_probs, best_paths, prep, states)

In [ ]:
s = 'Sundeep is topper'
s2 = 'Shibainu is trading all time high'
p = s.split()
lst = []
for i in s.split():
  if i not in vocab: 
    word = assign_unk(i)
    lst.append(word)
  else:
    lst.append(i)
lst

['--unk_upper--', 'is', '--unk_noun--']

In [ ]:
best_probs, best_paths = initialize(states, tag_counts, A, B, lst, vocab)
best_probs, best_paths = viterbi_forward(A, B, lst, best_probs, best_paths, vocab)
pred = viterbi_backward(best_probs, best_paths, lst, states)

In [ ]:
#pred = viterbi_backward(best_probs, best_paths, p, states)
for i in range(len(lst)):
  print(pred[i], '---', lst[i])

NNP --- --unk_upper--
VBZ --- is
`` --- --unk_noun--


In [ ]:
final = []
actual = []
for line in test_corpus:
  wrd, tag = get_word_tag(line, vocab)
  final.append(wrd)
  actual.append(tag)

In [ ]:
best_probs, best_paths = initialize(states, tag_counts, A, B, final, vocab)
best_probs, best_paths = viterbi_forward(A, B, final, best_probs, best_paths, vocab)
pred = viterbi_backward(best_probs, best_paths, final, states)

In [ ]:
len(pred), len(actual)

(34199, 34199)

In [ ]:
cnt = 0
for i in range(len(pred)):
  if pred[i]==actual[i]:
    cnt+=1
acc = cnt/len(pred)
acc

0.9562852714991666

In [ ]:
s = 'GM Profit Sinks as Chip Shortage Takes Toll'
s1 = s.lower()
final = []
for wrd in s1.split():
  if wrd not in vocab: 
    word = assign_unk(wrd)
    final.append(word)
  else:
    final.append(wrd)
final

['--unk--', 'profit', '--unk--', 'as', 'chip', 'shortage', 'takes', 'toll']

In [ ]:
best_probs, best_paths = initialize(states, tag_counts, A, B, final, vocab)
best_probs, best_paths = viterbi_forward(A, B, final, best_probs, best_paths, vocab)
pred = viterbi_backward(best_probs, best_paths, final, states)

In [ ]:
for i in range(len(pred)):
  print(s1.split()[i],'---------',pred[i])

gm --------- JJ
profit --------- NN
sinks --------- NNS
as --------- IN
chip --------- NN
shortage --------- NN
takes --------- VBZ
toll --------- ``


In [ ]:
gm NNP
profit NN
sinks NNS
as IN
chip NN
shortage NN
takes VBZ
toll NN

In [ ]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(s)

In [ ]:
for ent in doc:
  print(ent.text, ent.tag_)

GM NNP
Profit NNP
Sinks NNP
as IN
Chip NN
Shortage NN
Takes VBZ
Toll NN


In [ ]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4 MB 1.2 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051302 sha256=4a9c5eb46f9a3b50b5f3e0c0cc6d3e1f1f8fab032969fc3b8ec0583a3f8ebdf0
  Stored in directory: /tmp/pip-ephem-wheel-cache-wwuutnhu/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
